In [1]:
import ipywidgets as widgets
import numpy as np
from ase import Atoms
from ase.cluster import wulff_construction
from abtem import Probe, Waves
from abtem.visualize.interactive.artists import ImageArtist, LinesArtist, MeasurementArtist1d, MeasurementArtist2d
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.utils import domain_coloring
from abtem.visualize.interactive.utils import throttle
from traitlets import link
from ase.build import graphene
from abtem.structures import orthogonalize_cell
from abtem import PlaneWave, CTF
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))


In [2]:
# atoms = orthogonalize_cell(graphene())
# atoms.center(axis=2, vacuum=2)
# atoms *= (3,2,1)

# wave = PlaneWave(energy=300e3, sampling=.1)
# graphene_exit_wave = wave.multislice(atoms, pbar=False)

# graphene_exit_wave.write('graphene_exit_wave.hdf5')

graphene_exit_wave = Waves.read('data/graphene_exit_wave.hdf5')

In [3]:
# surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)] # Nanoparticle facets
# esurf = [1.0, 1.1, 0.9] # Relative surface energies
# size = 1000 # Target number of atoms
# atoms = Atoms(wulff_construction('Au', surfaces, esurf, size, 'fcc', rounding='above'))
# atoms.center(vacuum=5)
# atoms.rotate(45, 'x', center='cop')

# wave = PlaneWave(energy=300e3, sampling=.1)
# nanoparticle_exit_wave = wave.multislice(atoms, pbar=False)

# nanoparticle_exit_wave.write('nanoparticle_exit_wave.hdf5')

nanoparticle_exit_wave = Waves.read('data/nanoparticle_exit_wave.hdf5')

In [18]:
max_semiangle = 100

canvas1 = Canvas()
canvas2 = Canvas()

ctf_artist = MeasurementArtist1d(colors='red')
envelope_artist = MeasurementArtist1d(colors='blue')
aperture_artist = MeasurementArtist1d(colors='green')
temporal_envelope_artist = MeasurementArtist1d(colors='orange')
spatial_envelope_artist = MeasurementArtist1d(colors='magenta')
gaussian_envelope_artist = MeasurementArtist1d(colors='cyan')

canvas1.artists = {'ctf': ctf_artist, 
                   'envelope': envelope_artist, 
                   'aperture':aperture_artist, 
                   'temporal_envelope':temporal_envelope_artist,
                   'spatial_envelope':spatial_envelope_artist,
                   'gaussian_envelope':gaussian_envelope_artist
                  
                  
                  }
canvas1.y_scale.min = -1.1
canvas1.y_scale.max = 1.1

artist2 = MeasurementArtist2d()
canvas2.artists = {'image': artist2}

defocus_slider = widgets.FloatSlider(description='Defocus', min=-1e2, max=1e2, value=0, step=1e1)
cs_slider = widgets.FloatSlider(description='Cs', min=-1e5, max=1e5, value=0, step=1e3)
aperture_slider = widgets.FloatSlider(description='Aperture', min=10, max=100, value=30, step=1)
focal_spread_slider = widgets.FloatSlider(description='Focal spread', min=0, max=50, value=0, step=1)
angular_spread_slider = widgets.FloatSlider(description='Angular spread', min=0, max=50, value=0, step=1)
gaussian_spread_slider = widgets.FloatSlider(description='Gaussian spread', min=0, max=2, value=0, step=.1)

structure_dropdown = widgets.Dropdown(description='Structure', options=['Graphene', 'Nanoparticle'], value='Graphene')

def update(*args):
    if structure_dropdown.value == 'Graphene':
        exit_wave = graphene_exit_wave
    elif structure_dropdown.value == 'Nanoparticle':
        exit_wave = nanoparticle_exit_wave
    
    ctf = CTF(defocus = defocus_slider.value,
              Cs = cs_slider.value,
              semiangle_cutoff=aperture_slider.value,
              energy=exit_wave.energy,
              focal_spread=focal_spread_slider.value,
              angular_spread = angular_spread_slider.value,
              gaussian_spread = gaussian_spread_slider.value
             )
    
    profiles = ctf.profiles(max_semiangle, phi=0)
    aberrated_wave = exit_wave.apply_ctf(ctf)
    
    for name, artist in canvas1.artists.items():
        
        artist.measurement = profiles[name]

    artist2.measurement = aberrated_wave.intensity()[0]

defocus_slider.observe(update, 'value')
cs_slider.observe(update, 'value')
aperture_slider.observe(update, 'value')
focal_spread_slider.observe(update, 'value')
angular_spread_slider.observe(update, 'value')
gaussian_spread_slider.observe(update, 'value')

structure_dropdown.observe(update, 'value')

autoadjust_button = widgets.ToggleButton(description='Autoadjust colorscale', value=True)
link((autoadjust_button, 'value'), (artist2._image_artist, 'autoadjust_colorscale'));


ctf_artist._lines_artist._mark.labels = ['CTF']
envelope_artist._lines_artist._mark.labels = ['Envelope']
aperture_artist._lines_artist._mark.labels = ['Aperture']
temporal_envelope_artist._lines_artist._mark.labels = ['Temporal envelope']
spatial_envelope_artist._lines_artist._mark.labels = ['Spatial envelope']
gaussian_envelope_artist._lines_artist._mark.labels = ['Gaussian envelope']

ctf_artist._lines_artist._mark.display_legend = True
envelope_artist._lines_artist._mark.display_legend = True
aperture_artist._lines_artist._mark.display_legend = True
temporal_envelope_artist._lines_artist._mark.display_legend = True
spatial_envelope_artist._lines_artist._mark.display_legend = True
gaussian_envelope_artist._lines_artist._mark.display_legend = True

#linesartist2._mark.display_legend = True

canvas1.y_label = 'CTF'
canvas1.x_label = 'Scattering angle [mread]'

canvas2.x_label = 'x [Å]'
canvas2.y_label = 'y [Å]'

canvas1.figure.legend_location = 'bottom-right'

In [19]:
update()

widgets_box = widgets.VBox([structure_dropdown, aperture_slider, defocus_slider, cs_slider, 
                            focal_spread_slider, angular_spread_slider, gaussian_spread_slider, autoadjust_button])

widgets.HBox([canvas1.widget, canvas2.widget,
widgets_box
             
             ])